Generates tables involving Shannon Entropy and Mutual Information.

In [ ]:
import matplotlib.pyplot as plt
import math
import re
import os

# mount drive -- only applies if the dataset is in the drive. otherwise, point to relevant dataset
from google.colab import drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# drive.mount('/content/drive/')
#dir = '/content/drive/My Drive/CAS-Assignment1/GraphGeneration/simcov_data/default'
dir = '/content/drive/My Drive/CAS-Assignment1/GraphGeneration/simcov_data/fig3'


In [ ]:
filename = 'simcov_3.stats'

time = []
incb = []
expr = []
apop = []
dead = []
tvas = []
ttis = []
chem = []
virs = []
chempts = []
infct = []

def makeSimpleTimeSeries(x_init, r, numSteps):
  x_old = x_init
  xs = [x_init]

  for i in range(0, numSteps):
    x_new = r * x_old * (1 - x_old)
    x_old = x_new
    xs.append(x_new)

  return xs

def getShannonEntropy(relativePops, basePop, startInd=None, endInd=None):
  popToOccurences = {} # dict from population size to number of occurences

  if startInd is None:
    startInd = 0

  if endInd is None:
    endInd = len(relativePops) - 1

  numPops = endInd - startInd + 1

  for i in range(startInd, endInd, 1):
    pop = math.floor(relativePops[i] * basePop)

    if pop in popToOccurences:
      popToOccurences[pop] = popToOccurences[pop] + 1
    else:
      popToOccurences[pop] = 1

  sum = 0

  for pop, occurenceCount in popToOccurences.items():
    probability = occurenceCount / numPops

    sum = sum + probability * math.log2(probability)

  return -sum


def getMutualInformation(relativePops1, relativePops2, basePop, startInd=None, endInd=None):
  popToOccurences = {} # dict from population size to { [1] = occurences1, [2] = occurences2 }

  if startInd is None:
    startInd = 0

  if endInd is None:
    endInd = len(relativePops1) - 1

  numPops = endInd - startInd + 1

  for i in range(startInd, endInd, 1):
    pop1 = math.floor(relativePops1[i] * basePop)
    pop2 = math.floor(relativePops2[i] * basePop)

    if pop1 in popToOccurences:
      popToOccurences[pop1][1] = popToOccurences[pop1][1] + 1
    else:
      popToOccurences[pop1] = { 1:1, 2:0 }

    if pop2 in popToOccurences:
      popToOccurences[pop2][2] = popToOccurences[pop2][2] + 1
    else:
      popToOccurences[pop2] = { 1:0, 2:1 }

  sum = 0

  for pop, occurences in popToOccurences.items():
    prob1 = occurences[1] / numPops
    prob2 = occurences[2] / numPops

    # The two runs are independent, so P(A or B) = P(A) + P(B) - P(A and B) = P(A) + P(B) - P(A) * P(B)
    probJoint = prob1 + prob2 - prob1 * prob2

    # I(X;Y) = H(X) + H(Y) - H(X or Y)
    sum = sum +\
      prob1 * math.log2( prob1 if prob1 > 0 else 1 ) +\
      prob2 * math.log2( prob2 if prob2 > 0 else 1 ) -\
      probJoint * math.log2( probJoint )

  return -sum


# map incoming data to lists for plotting
with open(os.path.join(dir, filename), "r") as f:
  for line in f:
    if line.startswith('#'):
      continue
    data = line.split('\t')
    time.append(float(data[0]) / 1440)
    incb.append(float(data[1]))
    expr.append(float(data[2]))
    apop.append(float(data[3]))
    dead.append(float(data[4]))
    tvas.append(float(data[5]))
    ttis.append(float(data[6]))
    chem.append(float(data[7]))
    virs.append(float(data[8]))
    chempts.append(float(data[9]))
    infct.append(float(data[10]))


In [ ]:
numTimeSteps = 200
basePopulationSize = 100
divergentTimeStep = 25
x_init_a = 0.3
x_init_b = 0.300001
r_1 = 2.9 # Stable populations
r_2 = 3.5 # Periodic populations
r_3 = 3.7 # Chaotic populations


xs_1a = makeSimpleTimeSeries(x_init_a, r_1, numTimeSteps)
xs_1b = makeSimpleTimeSeries(x_init_b, r_1, numTimeSteps)

xs_2a = makeSimpleTimeSeries(x_init_a, r_2, numTimeSteps)
xs_2b = makeSimpleTimeSeries(x_init_b, r_2, numTimeSteps)

xs_3a = makeSimpleTimeSeries(x_init_a, r_3, numTimeSteps)
xs_3b = makeSimpleTimeSeries(x_init_b, r_3, numTimeSteps)


print("n = " + str(divergentTimeStep))
print()

print("H(1..n in chaotic_a) = " + str(getShannonEntropy(xs_3a, basePopulationSize, 1, divergentTimeStep)))
print("H(n+1..N in chaotic_a) = " + str(getShannonEntropy(xs_3a, basePopulationSize, divergentTimeStep + 1, numTimeSteps)))
print()

print("H(1..n in periodic_a) = " + str(getShannonEntropy(xs_2a, basePopulationSize, 1, divergentTimeStep)))
print("H(n+1..N in periodic_a) = " + str(getShannonEntropy(xs_2a, basePopulationSize, divergentTimeStep + 1, numTimeSteps)))
print()

print("MI(1..n in chaotic) = " + str(getMutualInformation(xs_3a, xs_3b, basePopulationSize, 1, divergentTimeStep)))
print("MI(n+1..N in chaotic) = " + str(getMutualInformation(xs_3a, xs_3b, basePopulationSize, divergentTimeStep + 1, numTimeSteps)))
print()

print("MI(1..n in periodic) = " + str(getMutualInformation(xs_2a, xs_2b, basePopulationSize, 1, divergentTimeStep)))
print("MI(n+1..N in periodic) = " + str(getMutualInformation(xs_2a, xs_2b, basePopulationSize, divergentTimeStep + 1, numTimeSteps)))
print()




n = 25

H(1..n in chaotic_a) = 4.058101942183733
H(n+1..N in chaotic_a) = 5.5377237043391725

H(1..n in periodic_a) = 3.8679064420971963
H(n+1..N in periodic_a) = 1.9966970406535007

MI(1..n in chaotic) = 1.9254592981307852
MI(n+1..N in chaotic) = 1.7970570990922057

MI(1..n in periodic) = 1.9972285087343387
MI(n+1..N in periodic) = 1.904264527927898



In [ ]:
basePopulationSize = 100 / max(max(expr), max(ttis))
startInd = None
endInd = None

print("H(expressing cells) = " + str(getShannonEntropy(expr, basePopulationSize, startInd, endInd)))
print("H(T cells) = " + str(getShannonEntropy(ttis, basePopulationSize, startInd, endInd)))
print("MI(expressing cells, T cells) = " + str(getMutualInformation(expr, ttis, basePopulationSize, startInd, endInd)))
print()

H(expressing cells) = 6.614456309192915
H(T cells) = 6.149726293386289e-05
MI(expressing cells, T cells) = 0.16223776285903707

